In [1]:
import preprocessing
import pandas as pd

In [2]:
titles_file = './anomaly-detection-competition-ml1-ts-spring-2020/docs_titles.tsv/docs_titles.tsv'
train_file = './anomaly-detection-competition-ml1-ts-spring-2020/train_groups.csv'
test_file = './anomaly-detection-competition-ml1-ts-spring-2020/test_groups.csv'

In [3]:
doc_to_title = preprocessing.get_page_dict(titles_file)

In [4]:
X, y, group_ids = preprocessing.tf_idf(doc_to_title, train_file)

In [33]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression

In [41]:
f_dict = {}

In [42]:
clf = GaussianNB()

In [44]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, train_file)
f_dict['gaussian_nb'] = f

preparing tf-idf matrices....
calculating scores....

threshold: 0.1 
f1: 0.71829386664516


In [45]:
gaussian_nb = preprocessing.predict_test(clf, th, doc_to_title, train_file, test_file)

preparing tf-idf matrices....


In [46]:
clf = BaggingClassifier(n_estimators=50, random_state=111)

In [47]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, train_file)
f_dict['bagging'] = f

preparing tf-idf matrices....
calculating scores....

threshold: 0.33333333333333337 
f1: 0.7043124927568255


In [48]:
bagging = preprocessing.predict_test(clf, th, doc_to_title, train_file, test_file)

preparing tf-idf matrices....


In [49]:
clf = GradientBoostingClassifier()

In [50]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, train_file)
f_dict['xgboost'] = f

preparing tf-idf matrices....
calculating scores....

threshold: 0.3666666666666667 
f1: 0.7171476005714545


In [51]:
xgboost = preprocessing.predict_test(clf, th, doc_to_title, train_file, test_file)

preparing tf-idf matrices....


In [52]:
clf = ExtraTreesClassifier(n_estimators=1000, random_state=111)

In [54]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, train_file)
f_dict['extratrees'] = f

preparing tf-idf matrices....
calculating scores....

threshold: 0.4 
f1: 0.7101494666404454


In [55]:
extratrees = preprocessing.predict_test(clf, th, doc_to_title, train_file, test_file)

preparing tf-idf matrices....


In [56]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000)

In [57]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, train_file)
f_dict['logistic'] = f

preparing tf-idf matrices....
calculating scores....

threshold: 0.30000000000000004 
f1: 0.719029473861281


In [58]:
logistic = preprocessing.predict_test(clf, th, doc_to_title, train_file, test_file)

preparing tf-idf matrices....


In [60]:
total_res = pd.DataFrame({'gaussian_nb':gaussian_nb,
              'bagging': bagging,
              'xgboost': xgboost,
              'extratrees': extratrees,
              'logistic': logistic})

In [64]:
total_res = total_res.applymap(lambda x: 1 if x else -1)

In [67]:
for i in total_res.columns:
    total_res.loc[:,i] *= f_dict.get(i)

In [78]:
res = (total_res.mean(axis=1) > 0) * 1

In [85]:
total_res = pd.read_csv(test_file)
total_res['target'] = res

In [87]:
total_res[['pair_id', 'target']].to_csv('lets_try_normal.csv', index=None)